In [1]:
from tensorflow import keras

## Data Augmention 을 통한 가위-바위-보 데이터 셋 분류 성능 개선

### Step 1. Input tensor 와 Target tensor 준비(훈련데이터)

(1) 가위-바위-보 데이터셋 다운로드

In [2]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'

path_to_zip = keras.utils.get_file('rps.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

200695808/200682221 [==============================] - 2s 0us/step


In [3]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'

path_to_zip = keras.utils.get_file('rps_test.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

29532160/29516758 [==============================] - 0s 0us/step


(2) ImageDataGenerator를 이용해 이미지 파일을 load 하기 위한 경로 지정

In [4]:
train_dir = '/content/datasets/rps'
test_dir = '/content/datasets/rps-test-set'

(3) ImageDataGenerator 객체 생성  
* 객체 생성 시 rescale 인자를 이용하여 텐서 내 원소의 범위를 [0 ~ 255] => [0 ~ 1] 로 ReScaling 진행
* Data Augmentation 을 위한 정보들 설정

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# Augmentation 수행 인자
# rotation : 40
# width : 0.2
# hight : 0.2
# share : 0.2
# zoom : 0.2
# horizontal : True
train_datagen = ImageDataGenerator(
    rescale=1./225.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

* .flow_from_directory() 메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator 객체 생성

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 2520 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


### Step 2. CNN 모델 디자인 및 학습 정보 설정

(1) CNN 모델 구성

In [8]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(3, activation='sigmoid'))

(2) 학습을 위한 설정 정보 지정

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Step 3. 모델에 데이터 generator 연결 후 학습 
  * model.fit() 이용하여 데이터 연결 및 학습시키기

In [10]:
history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=10,
      validation_data=validation_generator,
      validation_steps=len(validation_generator))

Epoch 1/10
79/79 [==============================] - 54s 305ms/step - loss: 1.3381 - accuracy: 0.4167 - val_loss: 0.9447 - val_accuracy: 0.4742
Epoch 2/10
79/79 [==============================] - 23s 291ms/step - loss: 0.8479 - accuracy: 0.6032 - val_loss: 0.4804 - val_accuracy: 0.8194
Epoch 3/10
79/79 [==============================] - 24s 297ms/step - loss: 0.5107 - accuracy: 0.8048 - val_loss: 0.3357 - val_accuracy: 0.8988
Epoch 4/10
79/79 [==============================] - 24s 300ms/step - loss: 0.3320 - accuracy: 0.8821 - val_loss: 0.1423 - val_accuracy: 0.9643
Epoch 5/10
79/79 [==============================] - 24s 297ms/step - loss: 0.2482 - accuracy: 0.9167 - val_loss: 0.7755 - val_accuracy: 0.5794
Epoch 6/10
79/79 [==============================] - 24s 300ms/step - loss: 0.1981 - accuracy: 0.9329 - val_loss: 0.0799 - val_accuracy: 0.9901
Epoch 7/10
79/79 [==============================] - 26s 324ms/step - loss: 0.1694 - accuracy: 0.9440 - val_loss: 0.0814 - val_accuracy: 0.9802

### Step 4. 테스트 데이터 셋을 통한 모델의 성능 평가

In [11]:
loss, accuracy = model.evaluate(test_generator)

12/12 [==============================] - 2s 146ms/step - loss: 0.0293 - accuracy: 0.9812
